In [ ]:
import salabim_plus as sim_plus
import salabim as sim
# import the different components here
import misc_tools
import datetime

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
runtime = 1000

with open('data/output_'+now+'.txt', 'w') as out:
    
    # step 1: create the simulation environment
    env = sim_plus.EnvironmentPlus(trace=out)
    
    # step 2: create the machines in the simulation environment
    # do there need to be things like assembly benches here?
    VSWR_CTI         =sim_plus.Machine(var_name='VSWR_CTI',         env=env)
    VSWR_CTI_DBG     =sim_plus.Machine(var_name='VSWR_CTI_DBG',     env=env)
    OPTX_CTI         =sim_plus.Machine(var_name='OPTX_CTI',         env=env)
    BFPD_CTI         =sim_plus.Machine(var_name='BFPD_CTI',         env=env)
    BFPD_CTI_DBG     =sim_plus.Machine(var_name='BFPD_CTI_DBG',     env=env)
    VSWR_CTI2        =sim_plus.Machine(var_name='VSWR_CTI2',        env=env)
    MOD_FEEDASSY_PAT =sim_plus.Machine(var_name='MOD_FEEDASSY_PAT', env=env)
    STATIC_CTI       =sim_plus.Machine(var_name='STATIC_CTI',       env=env)
    STATIC_CTI_DBG   =sim_plus.Machine(var_name='STATIC_CTI_DBG',   env=env)
    COND_EST         =sim_plus.Machine(var_name='COND_EST',         env=env)
    RF_FIT_TEST      =sim_plus.Machine(var_name='RF_FIT_TEST',      env=env)
    RF_CTG           =sim_plus.Machine(var_name='RF_CFG',           env=env)
    
    # step 3: make machine grouping for common processes
    # not sure what should and shouldn't go here
    # call sim_plus.MachineGroup()
    
    # step 4: create the workers in the simulation environment
    assembler        =sim_plus.Worker(var_name='assembler',    env=env, capacity=1)
    technician       =sim_plus.Worker(var_name='technician',   env=env, capacity=1)
    prod_control     =sim_plus.Worker(var_name='prod_control', env=env, capacity=1)
    inspection       =sim_plus.Worker(var_name='inspection',   env=env, capacity=1)
    
    # step 5: make the shift schedules for the workers
    
    # these probably need to be checked for correctness,
    # not sure if start_time=0 or if shift_type=pattern
    
    shift_schedule = misc_tools.make_shifts(shift_duration=24*60, 
                                            off_days=['saturday','sunday'])
    
    assembler_shift = sim_plus.ShiftController(worker=assembler,
                                               env=env,
                                               start_time=0,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    technician_shift = sim_plus.ShiftController(worker=technician,
                                               env=env,
                                               start_time=0,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    prod_control_shift = sim_plus.ShiftController(worker=prod_control,
                                               env=env,
                                               start_time=0,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    inspection_shift = sim_plus.ShiftController(worker=inspection,
                                               env=env,
                                               start_time=0,
                                               shifts=shift_schedule,
                                               shift_type='pattern')
    
    # step 6: point to the step creation function
    # call create_routing for the components here
    
    # step 7: make all of the generators that determine when entites are made
    # call sim_plus.EntityGenerator for each of the componenets
    
    # step 8: create all of the inventory locations in the simulation environment
    # call sim_plus.Kanban for each of the components with inventory systems
    
    # step 9: connect any entity boms and main_exits to its entity generator
    # e.g. part_c_gen.bom = part_c.get_bom(env=env.objs)
    #      part_a.main_exit = part_a_kanban
    
    # step 10: activate the controlling components within the simulation environment
    assembler_shift.activate(process='work')
    technician_shift.activate(process='work')
    prod_control_shift.active(process='work')
    inspection_shift.activate(process='work')
    
    # activate components
    # e.g. part_a_gen.activate(process='arrive')
    
    # stepp 11: execute the simulation
    env.run(till=runtime)